<a href="https://colab.research.google.com/github/lmcanavals/cs_topics/blob/main/0902_zebra_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ortools &> /dev/null

In [2]:
from ortools.sat.python import cp_model

#crear CSP
model = cp_model.CpModel()
#variables y dominios
##N = [English, Swedish, Danish, German, Norwegian]
N = [model.NewIntVar(   1,5,'N_'+str(i)) for i in range(5)]
##B = [Tea, Coffe, Milk, Beer, Water]
B = [model.NewIntVar(1,5,'N_'+str(i)) for i in range(5)]
##P = [Dog, Birds, Cats, Horse, Zebra]
P = [model.NewIntVar(1,5,'N_'+str(i)) for i in range(5)]
##C = [Red, Green, White, Blue, Yellow]
C = [model.NewIntVar(1,5,'N_'+str(i)) for i in range(5)]
##L = [PallMall, Dunhill, BlueMaster, Prince, Blend]
L = [model.NewIntVar(1,5,'N_'+str(i)) for i in range(5)]

#restricciones
##todos tienen atributos distintos
model.AddAllDifferent(N) #nacionalidades
model.AddAllDifferent(B) #bebidas
model.AddAllDifferent(P) #mascotas
model.AddAllDifferent(C) #colores
model.AddAllDifferent(L) #cigarros

#función para determinar si la variable x1 tomar el valor v1 si y solo si la variable x2 toma el valor v2 en una de las casas
def exists(model, x1, v1, x2, v2):
  ex = [model.NewBoolVar('e_'+str(j)) for j in range(5)]
  for i in range(5): #para cada casa
    #x1[i] vale v1
    ##ex[i] --> x1[i] == v1
    model.Add(x1[i] == v1).OnlyEnforceIf(ex[i])
    ##ex[i] <-- x1[i] == v1 <===> not(ex[i]) --> not(x1[i] == v1)
    model.Add(x1[i] != v1).OnlyEnforceIf(ex[i].Not())
    #x2[i] vale v2
    #(ex[i] <--> x2[i] == v2)
    #(ex[i] --> x2[i] == v2) and (ex[i] <-- x2[i] == v2)
    #(ex[i] --> x2[i] == v2) and (not(ex[i]) --> not(x2[i] == v2))
    #(ex[i] --> x2[i] == v2) and (not(ex[i]) --> x2[i] != v2)
    ##ex[i] --> x2[i] == v2
    model.Add(x2[i] == v2).OnlyEnforceIf(ex[i])
    ##ex[i] <-- x2[i] == v2 <===> not(ex[i]) --> not(x2[i] == v2)
    model.Add(x2[i] != v2).OnlyEnforceIf(ex[i].Not())
  #se deben cumplir ambas condiciones en exactamente 1 casa
  model.Add(sum(ex) == 1)

#función para determinar si la variable x1 tomar el valor v1 en una casa contigua a otra donde la variable x2 toma el valor v2
def neighbor(model, x1, v1, x2, v2):
  n = [model.NewBoolVar('n_'+str(j)) for j in range(4)]
  for i in range(4): #para cada casa con vecinos
    b = [model.NewBoolVar('b_'+str(j)) for j in range(2)]
    ##n[i] <--> (x1[i] == v1)
    model.Add(x1[i] == v1).OnlyEnforceIf(b[0])
    model.Add(x1[i] != v1).OnlyEnforceIf(b[0].Not())
    ##n[i] <--> (x2[i+1] == v2)
    model.Add(x2[i+1] == v2).OnlyEnforceIf(b[1])
    model.Add(x2[i+1] != v2).OnlyEnforceIf(b[1].Not())
    #model.Add(sum(b) == n[i])
    model.Add(sum(b) == 2).OnlyEnforceIf(n[i])
    model.Add(sum(b) != 2).OnlyEnforceIf(n[i].Not())
  return n

##The English man lives in the red house.
exists(model, N, 1, C, 1)
##The Swede has a dog.
exists(model, N, 2, P, 1)
##The Dane drinks tea.
exists(model, N, 3, B, 1)
##The green house is immediately to the left of the white house.
model.Add(sum(neighbor(model,C,2,C,3)) == 1)
##They drink coffee in the green house.
exists(model, B, 2, C, 2)
##The man who smokes Pall Mall has birds.
exists(model, L, 1, P, 2)
##In the yellow house they smoke Dunhill.
exists(model, C, 5, L, 2)
##In the middle house they drink milk.
model.Add(B[2] == 3)
##The Norwegian lives in the first house.
model.Add(N[0] == 5)
##The man who smokes Blend lives in the house next to the house with cats.
model.Add(sum(neighbor(model,L,5,P,3)) + sum(neighbor(model,P,3,L,5)) == 1)
##In a house next to the house where they have a horse, they smoke Dunhill.
model.Add(sum(neighbor(model,L,2,P,4)) + sum(neighbor(model,P,4,L,2)) == 1)
##The man who smokes Blue Master drinks beer.
exists(model, L, 3, B, 4)
##The German smokes Prince.
exists(model, N, 4, L, 4)
##The Norwegian lives next to the blue house.
model.Add(sum(neighbor(model,N,5,C,4)) + sum(neighbor(model,C,4,N,5)) == 1)
##They drink water in a house next to the house where they smoke Blend.
model.Add(sum(neighbor(model,B,5,L,5)) + sum(neighbor(model,L,5,B,5)) == 1)

#solver
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
  print('N', [solver.Value(x) for x in N])
  print('B', [solver.Value(x) for x in B])
  print('P', [solver.Value(x) for x in P])
  print('C', [solver.Value(x) for x in C])
  print('L', [solver.Value(x) for x in L])
  Nz = ['English', 'Swedish', 'Danish', 'German', 'Norwegian']
  Bz = ['Tea', 'Coffe', 'Milk', 'Beer', 'Water']
  Pz = ['Dog', 'Birds', 'Cats', 'Horse', 'Zebra']
  Cz = ['Red', 'Green', 'White', 'Blue', 'Yellow']
  Lz = ['PallMall', 'Dunhill', 'BlueMaster', 'Prince', 'Blend']
  for i in range(5):
    print('En la casa {}:\t'.format(i+1),
          Nz[solver.Value(N[i]) - 1],'\t',
          Bz[solver.Value(B[i]) - 1],'\t',
          Pz[solver.Value(P[i]) - 1],'\t',
          Cz[solver.Value(C[i]) - 1],'\t',
          Lz[solver.Value(L[i]) - 1])

N [5, 3, 1, 4, 2]
B [5, 1, 3, 2, 4]
P [3, 4, 2, 5, 1]
C [5, 4, 1, 2, 3]
L [2, 5, 1, 4, 3]
En la casa 1:	 Norwegian 	 Water 	 Cats 	 Yellow 	 Dunhill
En la casa 2:	 Danish 	 Tea 	 Horse 	 Blue 	 Blend
En la casa 3:	 English 	 Milk 	 Birds 	 Red 	 PallMall
En la casa 4:	 German 	 Coffe 	 Zebra 	 Green 	 Prince
En la casa 5:	 Swedish 	 Beer 	 Dog 	 White 	 BlueMaster
